We want to have access to the Science Data Challenge data, that are stored on dCache storage that is accessible from our JupyterHub with 'ctadata' service

[<img src="platform.png" width="250"/>](image.png)


In [ ]:
import ctadata
ctadata.list_dir('cta/sdc-internal-data/')

The metadata of all DL3 

In [ ]:
from astropy.io import fits
from astropy.time import Time
from astropy.coordinates import SkyCoord

ctadata.fetch_and_save_file('cta/sdc-internal-data/obs-index.fits.gz')
hdul=fits.open('obs-index.fits.gz')
pointings=hdul[1].data
RA_pnts=pointings['RA_PNT']
DEC_pnts=pointings['DEC_PNT']
c_pnts=SkyCoord(RA_pnts,DEC_pnts,unit='degree')

DL3_files=pointings['EVENTS_FILENAME']
OBSIDs=pointings['OBS_ID']
Tstart_sec=pointings['TSTART']
Tstop_sec=pointings['TSTOP']
OBJECT=pointings['OBJECT']
date=pointings['DATE-OBS']
time=pointings['TIME-OBS']
Tstart=Time(date+'T'+time, format='isot', scale='utc').mjd 
date=pointings['DATE-END']
time=pointings['TIME-END']

.... and metadata on instrument response functions:

In [ ]:
ctadata.fetch_and_save_file('cta/sdc-internal-data/hdu-index.fits.gz')
hdul=fits.open('hdu-index.fits.gz')
IRFS=hdul[1].data
IRF_OBSID=IRFS['OBS_ID']
IRF_FILE_NAME=IRFS['FILE_NAME']
IRF_HDU_NAME=IRFS['HDU_NAME']

We are interested in a specific source (say, Mrk 501). We do a "simple cone search" to find CTAO pointings that have the source within 3 degrees from the pointing direction:

In [ ]:
c_s = SkyCoord.from_name('Mrk 501')
RA_s=c_s.icrs.ra.deg
DEC_s=c_s.icrs.dec.deg
seps=c_s.separation(c_pnts).deg
mask=(seps<3.)
DL3_flist=DL3_files[mask]
OBSIDlist=OBSIDs[mask]
seps=seps[mask]
RA_pnts=RA_pnts[mask]
DEC_pnts=DEC_pnts[mask]


Just in case, we create a copy of the directory tree with SDC data locally:


In [ ]:
import os

print(DL3_flist[0])
dir='products_SDC/Events/North'
os.makedirs(dir,exist_ok=True)

In [ ]:
for f in DL3_flist:
    ctadata.fetch_and_save_file('cta/sdc-internal-data/'+f)
    f=f.split('/')[-1]
    !mv {f} {dir}

The easiest thing to do is to create a "countmap mosaic" of all selected pointings in certain energy range, say above an energy threshold $E_{thr}$:

In [ ]:
from numpy import cos,pi
import numpy as np

E_thr=0.1

cdec=cos(DEC_s*pi/180.)
imsize=5.
pixsize=0.02
npix=int(2*imsize/pixsize+1)
ra_bins=np.linspace(RA_s-imsize/cdec,RA_s+imsize/cdec,npix+1)
dec_bins=np.linspace(DEC_s-imsize,DEC_s+imsize,npix+1)
mosaic=np.zeros((npix,npix))

for f in DL3_flist:
    hdul=fits.open(f)
    ev=hdul['EVENTS'].data
    ev_ra=ev['RA']
    ev_dec=ev['DEC']
    ev_en=ev['ENERGY']
    mask=ev_en>E_thr
    h=np.histogram2d(ev_ra[mask],ev_dec[mask],bins=[ra_bins,dec_bins])
    mosaic+=h[0]

Just, in case, the convention in astronomy to show Right Ascension (RA) increasing from right to left:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,8))
plt.imshow(np.flip(np.transpose(mosaic)),extent=(ra_bins[-1]+pixsize/cdec/2.,ra_bins[0]-pixsize/cdec/2.,dec_bins[0]-pixsize/2.,dec_bins[-1]+pixsize/2.),aspect=1/cdec)
plt.colorbar(label='Counts per pixel')
plt.xlabel('RA, degree')
plt.ylabel('DEC, degree')
plt.scatter([RA_s],[DEC_s],marker='x',color='white')


For the spectral extraction, we want to calculate excess over the background, using the telescope "wobbling":
[<img src="wobble.png" width="250"/>](wobble.png)

We will collect events within certain "aperture" of the radius $R_s$ around the source position and estimate the background from the same aperture in the opposite wobble positions with respect to the center of the camera.

We bin these events in logarithmic bins of reconstructed energy.

In [ ]:
from numpy import log10,sqrt

Emin=0.01    
Emax=100. 
NEbins=20

Erec_bins=np.logspace(log10(Emin),log10(Emax),NEbins+1)
Erec_min=Erec_bins[:-1]
Erec_max=Erec_bins[1:]
Erec_mean=sqrt(Erec_min*Erec_max)
lgErec_mean=log10(Erec_mean)
dErec=Erec_bins[1:]-Erec_bins[:-1]


In [ ]:
R_s=0.3
N_bkg=3

Cts_s=[]
Cts_b=[]
N_b=[]

for ind in range(len(DL3_flist)):
    pointing=DL3_flist[ind]
    hdul=fits.open(pointing)
    ev=hdul['EVENTS'].data
    ev_ra=ev['RA']
    ev_dec=ev['DEC']
    ev_en=ev['ENERGY']
    ev_coords=SkyCoord(ev_ra,ev_dec,unit='degree')
    sep_s=ev_coords.separation(c_s).deg
    mask=(sep_s<R_s)
    Cts_s.append(np.histogram(ev_en[mask],bins=Erec_bins)[0])
    tmp=0.*Cts_s[-1]

    dRA=RA_s-RA_pnts[ind]
    dDEC=DEC_s-DEC_pnts[ind]
    RA_b1=RA_pnts[ind]-dRA
    DEC_b1=DEC_pnts[ind]-dDEC
    c_b1=SkyCoord(RA_b1,DEC_b1,unit='degree')    
    sep_b1=ev_coords.separation(c_b1).deg
    mask=(sep_b1<R_s)
    tmp+=np.histogram(ev_en[mask],bins=Erec_bins)[0]
    RA_b2=RA_pnts[ind]-dDEC/cdec
    DEC_b2=DEC_pnts[ind]+dRA*cdec
    c_b2=SkyCoord(RA_b2,DEC_b2,unit='degree')    
    sep_b2=ev_coords.separation(c_b2).deg
    mask=(sep_b2<R_s)
    tmp+=np.histogram(ev_en[mask],bins=Erec_bins)[0]
    RA_b3=RA_pnts[ind]+dDEC/cdec
    DEC_b3=DEC_pnts[ind]-dRA*cdec
    c_b3=SkyCoord(RA_b3,DEC_b3,unit='degree')    
    sep_b3=ev_coords.separation(c_b3).deg
    mask=(sep_b3<R_s)
    tmp+=np.histogram(ev_en[mask],bins=Erec_bins)[0]
    Cts_b.append(tmp)


    Texp=hdul[1].header['LIVETIME']


This is the excess:

In [ ]:
Src=sum(Cts_s)-sum(Cts_b)/N_bkg
Src_err=sqrt(sum(Cts_s)+sum(Cts_b)/N_bkg**2)
plt.plot(Erec_mean,sum(Cts_s),label='signal region')
plt.plot(Erec_mean,sum(Cts_b)/N_bkg,label='background regions')
plt.errorbar(Erec_mean,Src,Src_err,label='excess',xerr=[Erec_mean-Erec_min,Erec_max-Erec_mean],linestyle='none')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('$E_{rec}$, TeV')
plt.ylabel('Counts')
plt.legend(loc='lower left')

To convert the excess count statistics into flux measurements we need to know the exposure, product of exposure time $T_{exp}$ on effective area $A_{eff}$. 
$A_{eff}$, known from MC simulations, is estimated as a function of "true" energy of MC events. However, we know only "reconstructed" energy of real events. We need a mapping between the "true" and "reconstructed" energy. This mapping is called "RMF" (redistribution matrix function"), or, otherwise, "energy dispersion"

In [ ]:
NEbins_true=150
Et_min=0.01
Et_max=100

Etrue_bins=np.logspace(log10(Et_min),log10(Et_max),NEbins_true+1)
Etrue_min=Etrue_bins[:-1]
Etrue_max=Etrue_bins[1:]
Etrue_mean=sqrt(Etrue_min*Etrue_max)
lgEtrue_mean=log10(Etrue_mean)
dEtrue=Etrue_bins[1:]-Etrue_bins[:-1]

In [ ]:
ind=1
resp_files='IRFS/fits/'+IRF_FILE_NAME[IRF_OBSID==OBSIDlist[ind]]
j=0
while(resp_files[j][10]!='P'):
    j+=1
resp_file=resp_files[j]
hdul=fits.open(resp_file)
RMF=hdul['ENERGY DISPERSION'].data
ENERG_LO=RMF['ENERG_LO'][0]
ENERG_HI=RMF['ENERG_HI'][0]
ENERG=sqrt(ENERG_LO*ENERG_HI)
MIGRA_LO=RMF['ENERG_LO'][0]
MIGRA_HI=RMF['ENERG_HI'][0]
MIGRA=(MIGRA_LO+MIGRA_HI)/2.
mask=RMF['THETA_LO'][0]<seps[ind]
ind_th=len(RMF['THETA_LO'][0][mask])-1
RMF_th=RMF['MATRIX'][0][ind_th]

#RMF_Erec=np.zeros((len(ENERG),len(Emeans)))
pdf_tmp=np.zeros((len(ENERG),NEbins))
for k in range(len(ENERG)):
    #print(ENERG[k])
    E=MIGRA*ENERG[k]
    dE=E[1:]-E[:-1]
    E=sqrt(E[1:]*E[:-1])
    dpdf_dE=sqrt(RMF_th[1:,k]*RMF_th[:-1,k])/dE # this is differential PDF to have count in bin E (reconstructed) for true energy ENERG[k] 
    pdf_tmp[k]=np.interp(Erec_mean,E,dpdf_dE)*dErec
pdf=np.zeros((NEbins_true,NEbins))
for k in range(NEbins):
    pdf[:,k]=np.interp(Etrue_mean,ENERG,pdf_tmp[:,k])


In [ ]:
RMF['THETA_LO'],RMF['THETA_HI'],seps[ind],ind_th

In [ ]:
seps[ind]

The 2d Gaussian with normalisation SCALE perhaps looks like this
$$
PSF(\theta)=SCALE\cdot \exp\left(-\frac{\theta^2}{2\sigma^2}\right)
$$
The integral is 
$$
2\pi\int PSF(\theta) \theta d\theta=2\pi SCALE\sigma^2\int  \exp\left(-\frac{\theta^2}{2\sigma^2}\right) d\left(\frac{\theta^2}{2\sigma^2}\right)=2\pi\sigma^2 SCALE
$$
If we are considering only counts within certain radius, the "containment factor" is
$$
CF=2\pi SCALE\sigma^2\int_0^{R_s^2/2\sigma^2}  \exp\left(-\frac{\theta^2}{2\sigma^2}\right) d\left(\frac{\theta^2}{2\sigma^2}\right)=2\pi\sigma^2 SCALE\left(1-\exp\left(-\frac{R_s^2}{2\sigma^2}\right)\right)
$$


In [ ]:
psf=hdul['POINT SPREAD FUNCTION'].data
ENERG_LO=psf['ENERG_LO'][0]
ENERG_HI=psf['ENERG_HI'][0]
ENERG=sqrt(ENERG_LO*ENERG_HI)
mask=psf['THETA_LO'][0]<seps[ind]
ind_th=len(psf['THETA_LO'][0][mask])-1
print(ind_th)
#psf['SCALE']
#psf['SIGMA_1']
#psf['SIGMA_2']
#psf['AMPL_2']
#psf['AMPL_3']
psf_scale=psf['SCALE'][0][ind_th]
psf_sigma=psf['SIGMA_1'][0][ind_th]
psf_CF=2*pi*psf_sigma*psf_scale*(1-exp(-R_s**2/2/psf_sigma**2))
plt.plot(ENERG,psf_scale)
plt.plot(ENERG,psf_sigma)
plt.plot(ENERG,psf_CF)
plt.plot(ENERG,psf['SIGMA_2'][0][ind_th])
plt.plot(ENERG,psf['SIGMA_3'][0][ind_th])
plt.plot(ENERG,psf['AMPL_2'][0][ind_th])
plt.xscale('log')
plt.yscale('log')
print(psf.columns)
print(R_s)
#PSF looks like single 2d Gaussian of the width SIGMA_1, with normalisation given by SCALE. 
#The other two Gaussians seem to have zero width and amplitude.....

In [ ]:
pdf.shape
i=110
plt.step(Erec_bins,np.concatenate(([pdf[i,0]],pdf[i])))
plt.xscale('log')
plt.xlabel('$E_{rec}$, TeV')
plt.ylabel('probability to find $E_{rec}$ between $E_{rec}$ and $E_{rec}+\Delta E_{rec}$')
plt.text(1e-2,0.6,r'$E_{true}=$'+str(Etrue_mean[i])+' TeV')
plt.axvline(Etrue_mean[i],color='black',linestyle='dashed')
sum(pdf[i])


In [ ]:
pdf.shape
i=25
plt.step(Erec_bins,np.concatenate(([pdf[i,0]],pdf[i])))
plt.xscale('log')
plt.xlabel('$E_{rec}$, TeV')
plt.ylabel('probability to find $E_{rec}$ between $E_{rec}$ and $E_{rec}+\Delta E_{rec}$')
plt.text(1e-2,0.14,r'$E_{true}=$'+str(Etrue_mean[i])+' TeV')
plt.axvline(Etrue_mean[i],color='black',linestyle='dashed')
sum(pdf[i])

In [ ]:
pdf.shape
i=37
plt.step(Erec_bins,np.concatenate(([pdf[i,0]],pdf[i])))
plt.xscale('log')
plt.xlabel('$E_{rec}$, TeV')
plt.ylabel('probability to find $E_{rec}$ between $E_{rec}$ and $E_{rec}+\Delta E_{rec}$')
plt.text(1e-2,0.14,r'$E_{true}=$'+str(Etrue_mean[i])+' TeV')
plt.axvline(Etrue_mean[i],color='black',linestyle='dashed')
sum(pdf[i])

In [ ]:
RMFs=[]
Exposures=[]
Responses=[]
PSF_CFs=[]
for ind in range(len(DL3_flist)):
    resp_files='IRFS/fits/'+IRF_FILE_NAME[IRF_OBSID==OBSIDlist[ind]]
    j=0
    while(resp_files[j][10]!='P'):
        j+=1
    resp_file=resp_files[j]
    hdul=fits.open(resp_file)
    RMF=hdul['ENERGY DISPERSION'].data
    ENERG_LO=RMF['ENERG_LO'][0]
    ENERG_HI=RMF['ENERG_HI'][0]
    ENERG=sqrt(ENERG_LO*ENERG_HI)
    MIGRA_LO=RMF['ENERG_LO'][0]
    MIGRA_HI=RMF['ENERG_HI'][0]
    MIGRA=(MIGRA_LO+MIGRA_HI)/2.
    mask=RMF['THETA_LO'][0]<seps[ind]
    ind_th=len(RMF['THETA_LO'][0][mask])-1
    RMF_th=RMF['MATRIX'][0][ind_th]
    
    #RMF_Erec=np.zeros((len(ENERG),len(Emeans)))
    pdf_tmp=np.zeros((len(ENERG),NEbins))
    for k in range(len(ENERG)):
        #print(ENERG[k])
        E=MIGRA*ENERG[k]
        dE=E[1:]-E[:-1]
        E=sqrt(E[1:]*E[:-1])
        dpdf_dE=sqrt(RMF_th[1:,k]*RMF_th[:-1,k])/dE # this is differential PDF to have count in bin E (reconstructed) for true energy ENERG[k] 
        pdf_tmp[k]=np.interp(Erec_mean,E,dpdf_dE)*dErec
    pdf=np.zeros((NEbins_true,NEbins))
    for k in range(NEbins):
        pdf[:,k]=np.interp(Etrue_mean,ENERG,pdf_tmp[:,k])
    RMFs.append(pdf)
    
    AEFF=hdul['EFFECTIVE AREA'].data
    ENERG_A_LO=AEFF['ENERG_LO'][0]
    ENERG_A_HI=AEFF['ENERG_HI'][0]
    ENERG_A=sqrt(ENERG_A_LO*ENERG_A_HI)
    Effarea=AEFF['EFFAREA'][0][ind_th]
    AT=np.interp(Etrue_mean,ENERG_A,Effarea)*Texp*1e4


    psf=hdul['POINT SPREAD FUNCTION'].data
    ENERG_LO=psf['ENERG_LO'][0]
    ENERG_HI=psf['ENERG_HI'][0]
    ENERG=sqrt(ENERG_LO*ENERG_HI)
    mask=psf['THETA_LO'][0]<seps[ind]
    ind_th=len(psf['THETA_LO'][0][mask])-1
    psf_scale=psf['SCALE'][0][ind_th]
    psf_sigma=psf['SIGMA_1'][0][ind_th]
    psf_CF=2*pi*psf_sigma*psf_scale*(1-exp(-R_s**2/2/psf_sigma**2))
    tmp=np.interp(Etrue_mean,ENERG,psf_CF)
    Exposures.append(AT*tmp)
    response=np.outer(Exposures[0],np.ones(NEbins))*RMFs[0]
    Responses.append(response)


In [ ]:
plt.plot(Etrue_mean,sum(Exposures))
plt.xscale('log')
plt.yscale('log')
plt.xlabel('$E_{true}$, TeV')
plt.ylabel('$A_{eff}T_{exp}$, $cm^2s$')

In [ ]:
print(log10(Et_min),log10(Et_max),log10(Emin),log10(Emax))
plt.imshow(np.transpose(RMFs[-1]),extent=(log10(Et_min),log10(Et_max),log10(Emin),log10(Emax)),origin='lower')
plt.colorbar(label='pdf')
plt.xlabel('$log10(E_{true}/TeV)$')
plt.ylabel('$log10(E_{rec}/TeV)$')
x=np.linspace(-2,2)
plt.plot(x,x,color='white')

Above $E_{true}\sim 30$ GeV the RMF is diagonal, so the reconstructed energy is a good proxy for the true energy. in such situation, we can make a simple estimate of the flux in each true/reconstructed energy bin 
$$
E^2\frac{dN}{dE}=\frac{Cts_{src} E_{min}E_{max}}{(E_{max}-E_{min})}
$$
The data is reasonably well described by a broken powerlaw model, up to the energy $E\simeq 10$~TeV (in reality, absorption on EBL needs to be considered for better modelling)

In [ ]:
from numpy import exp
E0=1.
def bknpl_dNdE(E,N,Gam1,Gam2,Ebr,alpha):
    return N*(E/E0)**(-Gam1)/(1+(E/Ebr)**alpha)**((Gam2-Gam1)/alpha)
def cutoffpl_dNdE(E,N,Gam,Ecut,alpha):
    return N*(E/E0)**(-Gam)*exp(-(E/Ecut)**alpha)

Resp=sum(Responses)
Exposure=sum(Exposures)
Exposure_rec=np.interp(Erec_mean,Etrue_mean,Exposure)
factor=1/Exposure_rec/(Erec_max-Erec_min)*Erec_min*Erec_max
flux_simple=Src*factor
flux_simple_err=Src_err*factor
plt.errorbar(Erec_mean,flux_simple,flux_simple_err)
plt.xscale('log')
plt.yscale('log')

Norm=1.27e-11
Gam=2.13
Ecut=1/2.1370e-01	
alpha=1
y=cutoffpl_dNdE(Erec_mean,Norm,Gam,Ecut,alpha)
plt.plot(Erec_mean,y*Erec_mean**2)
plt.ylim(1e-13,1e-10)
plt.axvline(10**(-1.5),color='black',linestyle='dashed')
plt.xlabel('$E$, TeV')
plt.ylabel('$E^2 dN/dE$, TeV/cm$^2$s')

In [ ]:
Efit_min=10**(-1.5)
Efit_max=10.
def cutoffpl_counts(p):
    N,Gam,Ecut,alpha = p
    model=cutoffpl_dNdE(Etrue_mean,N,Gam,Ecut,alpha)*(Etrue_max-Etrue_min)
    return np.matmul(model,Resp)

Norm=1.27e-11
Gam=2.13
Ecut=1/2.1370e-01	
alpha=1
m=cutoffpl_counts([Norm,Gam,Ecut,alpha])
plt.plot(Erec_mean,m)
plt.errorbar(Erec_mean,Src,Src_err)
plt.axvline(Efit_min,color='black',linestyle='dashed')
plt.axvline(Efit_max,color='black',linestyle='dashed')
plt.xscale('log')
plt.yscale('log')
plt.ylim(0.1,1e5)

In [ ]:
Syst=0.05

def chisq(p):
    model=cutoffpl_counts(p)
    return (((model - Src)/(sqrt(Src_err**2+(Syst*Src)**2)+1e-20))[mask]**2).sum()
def chisq_log(p):
    p=[10**p[0],p[1],10**p[2],1]
    model=cutoffpl_counts(p)
    return (((model - Src)/(sqrt(Src_err**2+(Syst*Src)**2)+1e-20))[mask]**2).sum()

mask = Erec_mean > Efit_min
mask &= Erec_mean < Efit_max
mask &= Src_err>0.
dof=sum(mask)
print(dof)
#1.e-11,1.5,3.5,0.3
#Norm,Gam,Ecut,alpha
f = minimize(chisq_log, (log10(Norm),Gam,log10(Ecut)),method='Powell')
print(f)
N_best=10**(f.x[0])
Gam_best=f.x[1]
Ecut_best=10**f.x[2]
print(N_best,Gam_best,Ecut_best)

In [ ]:
plt.errorbar(Erec_mean,Src,Src_err)

model_best=cutoffpl_counts([N_best,Gam_best,Ecut_best,alpha_best])
plt.plot(Erec_mean,model_best)
excess=(Src-model_best)/model_best
excess_err=Src_err/model_best

plt.axvline(Efit_min,color='black',linestyle='dashed')
plt.axvline(Efit_max,color='black',linestyle='dashed')

plt.xscale('log')
plt.yscale('log')
plt.ylim(0.1,1e5)

In [ ]:
m=cutoffpl_dNdE(Erec_mean,N_best,Gam_best,Ecut_best,1)
plt.plot(Erec_mean,m*Erec_mean**2)
plt.errorbar(Erec_mean,flux_simple,flux_simple_err)
plt.xscale('log')
plt.yscale('log')
plt.ylim(1e-13,1e-10)